In [22]:
# Import Libraries
import sqlite3
import pandas as pd
import numpy as np
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
# Create database
db = sqlite3.connect("transactions.db")

# Connect to database
con = sqlite3.connect("/Users/DJ/E-Commerce-Recommendations/transactions.db")

In [3]:
# Load txt files into pandas and separate columns
adhesiveSealantTrans = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/Transactions with A&S.txt",sep='\t', header=0)
allTransaction = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/All Transations - 2 Weeks.txt", sep='\t', header=0)

In [4]:
# Check dataframes
adhesiveSealantTrans.head(5)

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [5]:
allTransaction.head(5)

,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603


## Exploratory  Analysis

### SQL Queries

In [6]:
# Move dataframes to sql tables in transactions.db
adhesiveSealantTrans.to_sql("aNs_transactions", con, if_exists="replace", index=False)
allTransaction.to_sql("all_trans", con, if_exists="replace", index=False)

In [7]:
# The case study states the data was "sanitized", but let's make sure
# Check aNs_transactions for missing data
print("Adhesive & Sealant Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))

# Check all_trans for missing data
print("\nAll Transactions Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))


Adhesive & Sealant Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0

All Transactions Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0


In [8]:
# See what items sell the most with same transaction as adhesives and sealants
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM ans_transactions GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 10;", con)

# Batteries are the most sold sku that are not in adhesives and sealants category, followed by hand wipes, gloves, and 
# paper towels. This makes batteries, cleaning supplies, and safety items the most purchased complements.

,l1,l2,l3,COUNT(sku)
0,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,33186
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,20845
2,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,18142
3,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,10161
4,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,9364
5,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,7910
6,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,7609
7,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,7060
8,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,6771
9,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,6494


In [9]:
# See what are top 10 items in ALL transactons
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM all_trans GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 10;", con)

# Batteries are most sold item, followed by paper cleaning products and air filters

,l1,l2,l3,COUNT(sku)
0,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,9234
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,7408
2,Safety,Footwear and Footwear Accessories,Socks,5721
3,Cleaning,Paper Products and Dispensers,"Paper Towels, Rolls",4965
4,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,4210
5,HVAC and Refrigeration,Air Filters,Pleated Air Filters,3261
6,HVAC and Refrigeration,Air Filters,Pleated Air Filters,3169
7,Cleaning,Paper Products and Dispensers,Toilet Paper,2955
8,HVAC and Refrigeration,Air Filters,Pleated Air Filters,2933
9,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,2831


### Pandas Exploration

In [10]:
# Check data types
print(" Adhesive & Sealant Data\n", adhesiveSealantTrans.dtypes)
print("\n All Transactions Data\n", allTransaction.dtypes)

 Adhesive & Sealant Data
 order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object

 All Transactions Data
 order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object


In [11]:
# See unique values for all features
print(" Adhesive & Sealant Data\n", adhesiveSealantTrans.nunique())
print("\n All Transactions Data\n", allTransaction.nunique())

 Adhesive & Sealant Data
 order_number    479917
l1                  33
l2                 586
l3                5771
sku             224423
brand             4296
dtype: int64

 All Transactions Data
 order_number    801575
l1                  33
l2                 593
l3                6203
sku             275958
brand             4574
dtype: int64


In [12]:
# Which Sku's are most popular?
aNs_skuCount = pd.DataFrame(adhesiveSealantTrans.groupby('sku')['order_number'].count())
print("Adhesives & Sealants\n", aNs_skuCount.sort_values('order_number', ascending=False).head())

all_skuCount = pd.DataFrame(allTransaction.groupby('sku')['order_number'].count())
print("\nAll Transactions\n", all_skuCount.sort_values('order_number', ascending=False).head())

Adhesives & Sealants
         order_number
sku                 
100164         33186
282471         20845
282473         18142
265850         10161
309746          9364

All Transactions
         order_number
sku                 
282471          9234
282473          7408
154210          5721
184171          4965
282469          4210


In [13]:
# Which items are these?
aNsItems = adhesiveSealantTrans[['sku', 'l3']]
print("Adhesives & Sealants\n", aNsItems.head())

allItems = allTransaction[['sku', 'l3']]
print("\nAll Transactions\n", allItems.head())

print("\nTop 5 Adhesive & sealant Items")
print(aNsItems[aNsItems['sku']==100164].head(1))
print(aNsItems[aNsItems['sku']==282471].head(1))
print(aNsItems[aNsItems['sku']==282473].head(1))
print(aNsItems[aNsItems['sku']==265850].head(1))
print(aNsItems[aNsItems['sku']==309746].head(1))

print("\nTop 5 All Items")
print(allItems[allItems['sku']==282471].head(1))
print(allItems[allItems['sku']==282473].head(1))
print(allItems[allItems['sku']==154210].head(1))
print(allItems[allItems['sku']==184171].head(1))
print(allItems[allItems['sku']==282469].head(1))

#Batteries are most popular sku in both dataframes

Adhesives & Sealants
       sku                             l3
0  122821                        Putties
1  276172            Caulks and Sealants
2  110119  Concrete Cleaners and Sealers
3   71052                  Plate Casters
4  251285            Retaining Compounds

All Transactions
       sku                         l3
0  265105   Reciprocating Saw Blades
1  215839      Temporary Leak Repair
2  284756                 Thresholds
3   12579         Standard Batteries
4  194681  General Purpose AC Motors

Top 5 Adhesive & sealant Items
        sku                l3
100  100164  Electrical Tapes
       sku                  l3
84  282471  Standard Batteries
        sku                  l3
129  282473  Standard Batteries
        sku                 l3
485  265850  Pipe Sealant Tape
        sku                    l3
492  309746  Duct and Cloth Tapes

Top 5 All Items
        sku                  l3
473  282471  Standard Batteries
        sku                  l3
389  282473  Standard Batterie

## Building a recommendation system

Given the limited amount of data available, we will use collaborative filtering using the items in each order number to score the similarity of each item based on which ones are purchased together and generate recommendations based on the similarity score.

The recommendation system will be created with allTransactions dataframe and then tested with sku's from adhesiveSealantTrans dataframe to see what is recommended.

### Prepare Data

Group by order number and drop less than 10    

In [14]:
# Check shape before filtering rows
allTransaction.shape

(2107537, 6)

In [15]:
# Drop sku and brand because they do not have enough info to be useful in collaborative filtering
allTransaction.drop('sku', axis=1, inplace=True)
allTransaction.drop('brand', axis=1, inplace=True)

# Drop most broad categories and keep most granular
allTransaction.drop('l1', axis=1, inplace=True)
allTransaction.drop('l2', axis=1, inplace=True)

In [50]:
# Combine items into lists by order number
grouped = allTransaction.groupby('order_number')['l3'].apply(list)
grouped.head()

order_number
1                     [Steel-Toe Work Boots and Shoes]
2    [Bib Overalls, Traffic Safety Vests, Work Cove...
3                                          [Rainsuits]
4                     [Steel-Toe Work Boots and Shoes]
5                     [Steel-Toe Work Boots and Shoes]
Name: l3, dtype: object

In [54]:
# Check shape of 
grouped.shape

(801575,)

In [56]:
#filter out small orders
filtered_group = [x for x in grouped if len(x)>=10 ]
filtered_group

[['Bib Overalls',
  'Traffic Safety Vests',
  'Work Coveralls',
  'Bib Overalls',
  'Traffic Safety Vests',
  'Traffic Safety Vests',
  'Bib Overalls',
  'Headbands, Sweatbands and Cooling Headwear',
  'Hard Hats',
  'Hard Hat Accessories',
  'Traffic Warning Flags and Banners',
  'Traffic Warning Flags and Banners',
  'Bib Overalls',
  'Hard Hats',
  'High Visibility Shirts and Sweatshirts'],
 ['Disposable Towels and Wipes',
  'Degreasers',
  'Coil Cleaners',
  'Drain Cleaning Cables',
  'Multi-Bit Screwdrivers',
  'Hex and Torx Key Sets',
  'Cleaning Rags',
  'Disposable Gloves',
  'Drain Cleaning Cables',
  'Coil Cleaners',
  'Handheld Flashlights',
  'Headlamps',
  'Handheld Flashlights',
  'Folding Utility Knives'],
 ['Structural Framing System Acc.',
  'Marking and Warning Tapes',
  'Direct Drive Corrosion Resistant Fans',
  'Indoor Furniture Covers',
  'Ultrasonic Cleaners',
  'Parts',
  'Coated Gloves',
  'Corrugated Tubing Fittings',
  'Lubrication Container Accessories',
  'C

In [58]:
# One-hot encode data in pandas dataframe
te = TransactionEncoder()
te_ary = te.fit(filtered_group).transform(filtered_group)
transaction_group = pd.DataFrame(te_ary, columns=te.columns_)
transaction_group

,12 Volt Accessories,12-Point Flange Head Cap Screws,3-Ring Binder Accessories,3-Ring Binders,3.3 Inch Diameter Motors,4.4 Inch Diameter Motors,5 X 20mm Glass and Ceramic Fuses,5S Red Tag Stations,A/C Conduit Kits,A/C Mounting Pads,...,Worm Gear Clamps,Wrap-a-Round Tape Measures,Wrist Rests and Palm Supports,Wrist Supports and Wraps,Y Strainers,Yard Hydrants,Yoke Ends,Zone Valve Actuators,Zone Valves,pH Meters
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
# Find items with any support using apriori algorithm to show which have any possible relationship (will filter more later)
apriori(transaction_group, min_support=0.005, use_colnames=True)

,support,itemsets
0,0.032623,(Abrasive Cut-Off and Chop Wheels)
1,0.011760,(Abrasive Rolls)
2,0.021155,(Absorbent Pads)
3,0.006771,(Absorbent Rolls)
4,0.023034,(Adhesive Bandages)
5,0.006188,(Adhesives)
6,0.011533,(Adjustable Wrench Sets)
7,0.027472,(Adjustable Wrenches)
8,0.005475,(Aerosol Dusters)
9,0.008585,(Air Freshener Refills)


In [65]:
# Find frequent item sets and and number of items in each order
freq_itemsets = apriori(transaction_group, min_support=0.01, use_colnames=True)
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x : len(x))
freq_itemsets.head()

,support,itemsets,length
0,0.032623,(Abrasive Cut-Off and Chop Wheels),1
1,0.011760,(Abrasive Rolls),1
2,0.021155,(Absorbent Pads),1
3,0.023034,(Adhesive Bandages),1
4,0.011533,(Adjustable Wrench Sets),1


In [75]:
#filter out smaller orders orders
freq_itemsets[ (freq_itemsets['length'] > 1) &
             (freq_itemsets['support'] > 0.02) ]

,support,itemsets,length
242,0.031359,"(Black Pipe Fittings, Black Pipe)",2
253,0.025593,"(Flat Washers, Cable Ties)",2
254,0.021446,"(Cable Ties, Hex Head Cap Screws)",2
260,0.021090,"(Cable Ties, Standard Batteries)",2
263,0.021576,"(Coated Gloves, Chemical Resistant Gloves)",2
264,0.029351,"(Cut-Resistant Gloves, Chemical Resistant Gloves)",2
265,0.026694,"(Disposable Gloves, Chemical Resistant Gloves)",2
269,0.037968,"(Chemical Resistant Gloves, Safety Glasses)",2
276,0.040430,"(Coated Gloves, Cut-Resistant Gloves)",2
278,0.036186,"(Disposable Gloves, Coated Gloves)",2


## Create recommendation engine

In [ ]:
# Create a similarity matrix using 1 - hamming to calculate jaccard similarity


In [ ]:
# Score "


In [ ]:
# Save

In [ ]:
# Print